In [1]:
import os
import pandas as pd
import json

# Specify the folder containing JSONL files
folder_path = '../Data/new_source_data/processing/gpt'
output_csv_path = '../Data/new_source_data/output/gpt_explaination_with_id.csv'

# Initialize a list to store the custom_id and content values from all files
records = []

# Loop through all files in the folder
for filename in os.listdir(folder_path):
    if filename.endswith('.jsonl'):
        file_path = os.path.join(folder_path, filename)
        
        # Read each JSONL file and extract custom_id and content values
        with open(file_path, 'r') as file:
            for line in file:
                record = json.loads(line)
                if 'custom_id' in record and 'response' in record:
                    custom_id_value = record['custom_id']
                    content_value = record['response'].get('body', {}).get('choices', [{}])[0].get('message', {}).get('content', '')
                    # Extract only the numerical part of the custom_id value
                    if 'record_' in custom_id_value:
                        records.append({
                            'custom_id': custom_id_value.split('_')[-1],
                            'content': content_value
                        })

# Create a DataFrame from the extracted values
df = pd.DataFrame(records)

df = df.rename(columns={'custom_id': 'record_number', 'content': 'gpt_explanation'})

df = df.drop_duplicates(subset='record_number')

# Save the DataFrame to a CSV file
df.to_csv(output_csv_path, index=False)

print(f"Combined custom_id and content values saved to {output_csv_path}")


Combined custom_id and content values saved to ../Data/new_source_data/output/gpt_explaination_with_id.csv


In [ ]:
input_path_bug_fix_dataset = '../Data/new_source_data/bug_fix_dataset.csv'
gpt_match_dataset = '../Data/new_source_data/output/gpt_explaination_with_id.csv'

bug_fix_df = pd.read_csv(input_path_bug_fix_dataset)
gpt_id_match_df = pd.read_csv(gpt_match_dataset)
bug_fix_gpt_df = bug_fix_df.merge(gpt_id_match_df, on='record_number', how='left')

bug_fix_gpt_df.to_csv("../Data/new_source_data/bug_fix_gpt.csv", index=False)